원본 데이터가 항상 지저분해서
그걸 조회→필터→정렬→클리닝 순서로 해결

In [1]:
import pandas as pd

raw = [
    {"date":"2026-01-01", "time":"09:10", "store":"A", "menu":"Americano", "price":"4,500원", "qty":"2", "paid":"TRUE"},
    {"date":"2026/01/01", "time":"09:12", "store":"A", "menu":"Latte",     "price":"5000",   "qty":1,   "paid":"True"}, # /
    {"date":"2026-01-02", "time":"12:30", "store":"A", "menu":"Latte",     "price":None,     "qty":2,   "paid":"FALSE"},
    {"date":"2026-01-03", "time":"18:05", "store":"B", "menu":"Mocha",     "price":"5500",   "qty":None,"paid":True},
    {"date":"2026-01-03", "time":"18:05", "store":"B", "menu":"Mocha",     "price":"5500",   "qty":None,"paid":True},  # 중복
    {"date":"2026-01-04", "time":"08:55", "store":"B", "menu":"Americano ", "price":"4500",  "qty":"1", "paid":"TRUE"}, # 공백
    {"date":"2026-01-04", "time":"08:58", "store":"A", "menu":"latte",     "price":"5,000",  "qty":"3", "paid":"TRUE"}, # 소문자
]
df = pd.DataFrame(raw)
df

,date,time,store,menu,price,qty,paid
0,2026-01-01,09:10,A,Americano,"4,500원",2,TRUE
1,2026/01/01,09:12,A,Latte,5000,1,True
2,2026-01-02,12:30,A,Latte,None,2,FALSE
3,2026-01-03,18:05,B,Mocha,5500,None,True
4,2026-01-03,18:05,B,Mocha,5500,None,True
5,2026-01-04,08:55,B,Americano,4500,1,TRUE
6,2026-01-04,08:58,A,latte,"5,000",3,TRUE


In [7]:
# 원본 데이터 결측치 조회
df.shape
df.info()
df.describe(include="all")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    7 non-null      object
 1   time    7 non-null      object
 2   store   7 non-null      object
 3   menu    7 non-null      object
 4   price   6 non-null      object
 5   qty     5 non-null      object
 6   paid    7 non-null      object
dtypes: object(7)
memory usage: 524.0+ bytes


,date,time,store,menu,price,qty,paid
count,7,7,7,7,6,5,7
unique,5,6,2,5,5,5,4
top,2026-01-04,18:05,A,Latte,5500,2,TRUE
freq,2,2,4,2,2,1,3


위에서 중복 행, 메뉴명 문제를 알아내는 것은 불가능하지 않은가?

In [11]:
# 인덱싱 핵심 .loc vs .iloc
# .loc = 라벨(이름) 기반 (끝 포함)
# .iloc = 위치(순서) 기반 (끝 미포함)
df.loc[0:2]

,date,time,store,menu,price,qty,paid
0,2026-01-01,09:10,A,Americano,"4,500원",2,TRUE
1,2026/01/01,09:12,A,Latte,5000,1,True
2,2026-01-02,12:30,A,Latte,None,2,FALSE


In [12]:
df.iloc[0:2]     # 0번째 위치

,date,time,store,menu,price,qty,paid
0,2026-01-01,09:10,A,Americano,"4,500원",2,TRUE
1,2026/01/01,09:12,A,Latte,5000,1,True


In [13]:
cols = ["date","time","store","menu","price","qty","paid"]
df2 = df[cols].copy() #데이터 굴려보려고 copy하는 것
df2.head()

,date,time,store,menu,price,qty,paid
0,2026-01-01,09:10,A,Americano,"4,500원",2,TRUE
1,2026/01/01,09:12,A,Latte,5000,1,True
2,2026-01-02,12:30,A,Latte,None,2,FALSE
3,2026-01-03,18:05,B,Mocha,5500,None,True
4,2026-01-03,18:05,B,Mocha,5500,None,True


In [14]:
# 데이터 처리
df2["paid"] = df2["paid"].astype(str).str.strip().str.lower()
df2["paid"].value_counts()

paid
true     6
false    1
Name: count, dtype: int64

In [15]:
# Boolean indexing으로 조건 생성
cond_paid = df2["paid"].isin(["true"])
cond_paid



0     True
1     True
2    False
3     True
4     True
5     True
6     True
Name: paid, dtype: bool

In [19]:
# 생성된 Boolean에서 True값 추출
paid_df = df2.loc[cond_paid]
paid_df

,date,time,store,menu,price,qty,paid
0,2026-01-01,09:10,A,Americano,"4,500원",2,true
1,2026/01/01,09:12,A,Latte,5000,1,true
3,2026-01-03,18:05,B,Mocha,5500,None,true
4,2026-01-03,18:05,B,Mocha,5500,None,true
5,2026-01-04,08:55,B,Americano,4500,1,true
6,2026-01-04,08:58,A,latte,"5,000",3,true


In [16]:
# time을 필터링하기 전에 미리 확인했어야하는 것.
df2["time"].apply(type).value_counts()

time
<class 'str'>    7
Name: count, dtype: int64

In [22]:
cond_storeA = (df2["store"] == "A")
cond_morning = df2["time"].str.startswith("09")
df_morning_A = df2.loc[cond_paid & cond_storeA & cond_morning, ["date","time","store","menu","price","qty"]]
df_morning_A

,date,time,store,menu,price,qty
0,2026-01-01,09:10,A,Americano,"4,500원",2
1,2026/01/01,09:12,A,Latte,5000,1


In [23]:
df2.sort_values(by=["date","time"], ascending=[True, True]).head()

,date,time,store,menu,price,qty,paid
0,2026-01-01,09:10,A,Americano,"4,500원",2,true
2,2026-01-02,12:30,A,Latte,None,2,false
3,2026-01-03,18:05,B,Mocha,5500,None,true
4,2026-01-03,18:05,B,Mocha,5500,None,true
5,2026-01-04,08:55,B,Americano,4500,1,true


In [24]:
# 클리닝 기본기 1 칼럼정리(통일)
df2 = df2.rename(columns={"qty":"quantity"})

In [25]:
# 클리닝 기본기 2 문자열 정리
df2["menu"] = df2["menu"].astype(str).str.strip().str.title()
df2["menu"].value_counts()
# str.title() 앞글자만 대문자로 바꿔준다.

menu
Latte        3
Americano    2
Mocha        2
Name: count, dtype: int64

In [27]:
# 클리닝 기본기 3 가격 정리
df2["price"] = (
    df2["price"]
    .astype(str)
    .str.replace(",", "", regex=False)
    .str.replace("원", "", regex=False)
    .str.strip()
)

df2["price"] = pd.to_numeric(df2["price"], errors="coerce")
df2["price"].isnull().sum()

np.int64(1)

In [28]:
df2["price"]

0    4500.0
1    5000.0
2       NaN
3    5500.0
4    5500.0
5    4500.0
6    5000.0
Name: price, dtype: float64

In [29]:
# 클리닝 기본기 4 dropna vs fillna (전략 선택)
# quantity도 숫자로 정리
df2["quantity"] = pd.to_numeric(df2["quantity"], errors="coerce")

# 핵심 변수 결측은 삭제(간단 버전)
clean = df2.dropna(subset=["price","quantity"]).copy()
clean.shape

(4, 7)

In [30]:
# 중복처리
clean.duplicated().sum()
clean = clean.drop_duplicates(keep="first")
clean.duplicated().sum()

np.int64(0)

In [31]:
# 결과물
clean["sales"] = clean["price"] * clean["quantity"]

meeting_table = clean.loc[:, ["date","time","store","menu","price","quantity","sales"]].sort_values(
    by=["date","store","time"],
    ascending=[True, True, True]
)

meeting_table

,date,time,store,menu,price,quantity,sales
0,2026-01-01,09:10,A,Americano,4500.0,2.0,9000.0
6,2026-01-04,08:58,A,Latte,5000.0,3.0,15000.0
5,2026-01-04,08:55,B,Americano,4500.0,1.0,4500.0
1,2026/01/01,09:12,A,Latte,5000.0,1.0,5000.0


In [32]:
meeting_table.to_csv("cafe_sales_clean_v1.csv", index=False, encoding="utf-8-sig")

In [51]:
import pandas as pd

raw = [
    {"date":"2026-01-01", "time":"09:10", "store":"A", "menu":"Americano", "price":"4,500원", "qty":"2", "paid":"TRUE"},
    {"date":"2026/01/01", "time":"09:12", "store":"A", "menu":"Latte",     "price":"5000",   "qty":1,   "paid":"True"}, # /
    {"date":"2026-01-02", "time":"12:30", "store":"A", "menu":"Latte",     "price":None,     "qty":2,   "paid":"FALSE"},
    {"date":"2026-01-03", "time":"18:05", "store":"B", "menu":"Mocha",     "price":"5500",   "qty":None,"paid":True},
    {"date":"2026-01-03", "time":"18:05", "store":"B", "menu":"Mocha",     "price":"5500",   "qty":None,"paid":True},  # 중복
    {"date":"2026-01-04", "time":"08:55", "store":"B", "menu":"Americano ", "price":"4500",  "qty":"1", "paid":"TRUE"}, # 공백
    {"date":"2026-01-04", "time":"08:58", "store":"A", "menu":"latte",     "price":"5,000",  "qty":"3", "paid":"TRUE"}, # 소문자
]
df = pd.DataFrame(raw)
df

cols = ["date","time","store","menu","price","qty","paid"]
df3= df[cols].copy()
df3["paid"] = df3["paid"].astype(str).str.strip().str.lower()

# paid true
paid_df3 = df3["paid"].isin(["true"])
# selecttrue = df3.loc[paid_df3]

# store A
storeA = df3["store"].isin(["A"])
# selectA = df3.loc[storeA]

# menu clean
df3["menu"]= df3["menu"].str.strip().str.title()

# quantity clean
df3 = df3.rename(columns={"qty":"quantity"})
df3['quantity']= df3['quantity'].astype(str).str.replace(',','',regex=False)
df3['quantity']= pd.to_numeric(df3['quantity'],errors='coerce')

# price clean
df3['price']= (
    df3['price'].astype(str)
    .str.replace(",", "", regex=False)
    .str.replace('원','',regex=False)
    .str.strip()
)
df3['price']= pd.to_numeric(df3['price'],errors='coerce')

# sales create
df3['sales']= df3['price']*df3['quantity']

# clean date, time
df3['date']= df3['date'].astype(str).str.strip().str.replace('/','-', regex=False)
df3['time']= df3['time'].astype(str).str.strip()
df3['wholedate']= pd.to_datetime(df3['date'] +" "+ df3['time'], errors='coerce')

# Clear duplicate
df3 = df3.drop_duplicates()

# result
meeting_table1 = df3.loc[paid_df3 & storeA, ["wholedate","store","menu","price","quantity","sales"]].sort_values(
    by=["wholedate","store"],
    ascending=[True, True]
)

meeting_table1

,wholedate,store,menu,price,quantity,sales
0,2026-01-01 09:10:00,A,Americano,4500.0,2.0,9000.0
1,2026-01-01 09:12:00,A,Latte,5000.0,1.0,5000.0
6,2026-01-04 08:58:00,A,Latte,5000.0,3.0,15000.0
